# DeltaStream Bronze Layer
---
Will perform the following in this stage:
- Read data from Kafka topics
- Convert data to delta tables

In [0]:
dbutils.widgets.text("Kafka Instance IP", "")
kafka_instance = dbutils.widgets.get("Kafka Instance IP")

Read data from Kafka topics and convert to Dataframes.

In [0]:
raw_airplane_stream = (spark.read
                            .format("kafka")
                            .option("kafka.bootstrap.servers", kafka_instance)
                            .option("subscribe", "airplane-topic")
                            .load())

raw_weather_stream = (spark.read
                            .format("kafka")
                            .option("kafka.bootstrap.servers", kafka_instance)
                            .option("subscribe", "weather-topic")
                            .load())

In [0]:
raw_airplane_df = raw_airplane_stream.selectExpr("CAST(value AS STRING)")
raw_weather_df  = raw_weather_stream.selectExpr("CAST(value AS STRING)")

Display dataframe data.

In [0]:
display(raw_airplane_df)

In [0]:
display(raw_weather_df)

Save as new bronze layer delta tables.

In [0]:
airplane_delta_df = (raw_airplane_df.write.format("delta")
                    .mode("append")
                    .options(mergeSchema=True) #enables evolving schemas
                    .saveAsTable("airplane_delta_bronze"))


weather_delta_df = (raw_weather_df.write.format("delta")
                    .mode("append")
                    .options(mergeSchema=True)
                    .saveAsTable("weather_delta_bronze"))